FastText 는 Facebook Research 에서도 제공하고 있습니다. 이는 pip install 로 이용 가능합니다.

```
pip install fasttext
```

학습의 편의를 위하여 Gensim 에서 제공하는 FastText 를 이용합니다. 한글의 FastText 를 적용하려면 초/중/종성을 분리해야 합니다. 이를 위하여 compose, decompose 함수를 이용합니다.

In [1]:
import config
from soynlp.hangle import decompose, compose
from soynlp.normalizer import remove_doublespace

def encode(s):
    def process(c):
        if c == ' ':
            return c
        jamo = decompose(c)
        # 'a' or 모음 or 자음
        if (jamo is None) or (jamo[0] == ' ') or (jamo[1] == ' '):
            return ' '
        base = jamo[0]+jamo[1]
        if jamo[2] == ' ':
            return base + '-'
        return base + jamo[2]

    s = ''.join(process(c) for c in s)
    return remove_doublespace(s).strip()

def decode(s):
    def process(t):
        assert len(t) % 3 == 0
        t_ = t.replace('-', ' ')
        chars = [tuple(t_[3*i:3*(i+1)]) for i in range(len(t_)//3)]
        recovered = [compose(*char) for char in chars]
        recovered = ''.join(recovered)
        return recovered

    return ' '.join(process(t) for t in s.split())

soynlp=0.0.49
added lovit_textmining_dataset


In [2]:
s = '가나다랄  a2ㅗㅛㅠ ㅋㅋㅋ 하핫'
print(encode(s))
print(decode(encode(s)))

ㄱㅏ-ㄴㅏ-ㄷㅏ-ㄹㅏㄹ ㅎㅏ-ㅎㅏㅅ
가나다랄 하핫


In [3]:
from navermovie_comments import get_movie_comments_path
from navermovie_comments import load_trained_embedding


class FastTextCorpus:
    def __init__(self, path):
        self.path = path
        self.n_iter = 0
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for i, doc in enumerate(f):
                if i % 10000 == 0 and i > 0:
                    print('\riter = {}, sents = {} ...'.format(self.n_iter, i), end='')
                idx, text, rate = doc.strip().split('\t')
                yield text.split()
            self.n_iter += 1
            print('\riter = {}, sents = {} done{}'.format(self.n_iter, i, ' '*20))

path = get_movie_comments_path(large=False, tokenize='fasttext')
corpus = FastTextCorpus(path)
for i, sent in enumerate(corpus):
    if i >= 3:
        break
    print(sent)

['ㅋㅡ-ㄹㅣ-ㅅㅡ-ㅌㅗ-ㅍㅓ-', 'ㄴㅗㄹㄹㅏㄴ', 'ㅇㅔ-ㄱㅔ-', 'ㅇㅜ-ㄹㅣ-ㄴㅡㄴ', 'ㄴㅗㄹㄹㅏㄴ', 'ㄷㅏ-']
['ㅇㅣㄴㅅㅔㅂㅅㅕㄴ', 'ㅈㅓㅇㅁㅏㄹ', 'ㅎㅡㅇㅁㅣ-ㅈㅣㄴㅈㅣㄴㅎㅏ-ㄱㅔ-', 'ㅂㅘㅆㅇㅓㅆㄱㅗ-', 'ㅋㅡ-ㄹㅣ-ㅅㅡ-ㅌㅗ-ㅍㅓ-', 'ㄴㅗㄹㄹㅏㄴ', 'ㄱㅏㅁㄷㅗㄱㄴㅣㅁ', 'ㅅㅣㄴㅈㅏㄱ', 'ㅇㅣㄴㅌㅓ-ㅅㅡ-ㅌㅔㄹㄹㅏ-ㄷㅗ-', 'ㅇㅣ-ㅂㅓㄴㅈㅜ-', 'ㅇㅣㄹㅇㅛ-ㅇㅣㄹㅇㅔ-', 'ㅂㅗ-ㄹㅓ-ㄱㅏㅂㄴㅣ-ㄷㅏ-', 'ㅇㅘㄴㅈㅓㄴ', 'ㄱㅣ-ㄷㅐ-ㅈㅜㅇ']
['ㄴㅗㄹㄹㅏㄴㅇㅣ-ㅁㅕㄴ', 'ㅁㅜ-ㅈㅗ-ㄱㅓㄴ', 'ㅂㅘ-ㅇㅑ-', 'ㄷㅚㄴㄷㅏ-', 'ㅇㅙ-ㄴㅑ-ㅎㅏ-ㅁㅕㄴ', 'ㅁㅗ-ㄷㅡㄴ', 'ㅈㅏㄱㅍㅜㅁㅇㅡㄹ', 'ㄷㅏ-', 'ㅎㅣ-ㅌㅡ-', 'ㅊㅕㅆㅇㅡ-ㄴㅣ-ㄲㅏㄴ']


시간이 오래 걸리기 때문에 모델을 미리 학습해뒀습니다.

In [4]:
import pickle
from gensim.models import FastText

TRAIN = False
if TRAIN:
    fasttext_model = FastText(
        corpus,
        window = 3,
        min_count = 10,
        min_n = 3,
        max_n = 6
    )
else:
    fasttext_model = load_trained_embedding(
        data_name = 'small',
        tokenize = 'fasttext',
        embedding = 'fasttext'
    )

한글로 이뤄진 단어 (query) 를 입력하면 preprocessing 을 통하여 초/중/종성으로 분해된 단어를 만듭니다. 이를 바탕으로 fasttext 에서 벡터가 비슷한 단어를 찾습니다.

fasttext model 에 학습된 단어는 초/중/종성이 분해된 형태입니다. 이를 우리가 이해하기 쉬운 형식으로 복원합니다.

In [5]:
def most_similar(query, topn=10):
    query_ = encode(query)
    similars = fasttext_model.wv.most_similar(query_, topn=topn)
    similars = [(decode(word), sim) for word, sim in similars]
    return similars

In [6]:
most_similar('영홯')

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('영활', 0.9843769073486328),
 ('영환', 0.9834325313568115),
 ('영화표', 0.9814302921295166),
 ('영화평', 0.9701882004737854),
 ('영화판', 0.9635608196258545),
 ('영화비', 0.9625624418258667),
 ('영화처럼', 0.9606270790100098),
 ('영화화', 0.9602814316749573),
 ('영화계', 0.9562531113624573),
 ('영화죠', 0.9556574821472168)]